In [22]:
!pip install fastText
!pip install datasets

You should consider upgrading via the '/home/victor/Documents/venv_epita/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/victor/Documents/venv_epita/bin/python -m pip install --upgrade pip' command.


In [23]:
import fasttext
from datasets import load_dataset

dataset_train = load_dataset('imdb', split='train')

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [24]:
import random
random.seed(42)

In [25]:
from string import punctuation


def to_lower(review):
	return str.lower(review)


def no_punct(review):
	return review.translate(str.maketrans('', '', punctuation))


def to_fasttext(review, label):

	label_str = 'positive'
	if label == 0:
		label_str = 'negative'
	return '__label__' + label_str + " " + review


def pretreatment(dataset):
	labels = dataset['label']
	reviews = dataset['text']
	corrected_review_list = []
	for i in range(len(reviews)):
		lower = to_lower(reviews[i])
		no_punc = no_punct(lower)
		corrected_review = to_fasttext(no_punc, labels[i])
		corrected_review_list.append(corrected_review)
	return corrected_review_list

def write_into_file(file_path: str, data: list):
	with open(file_path, mode="w") as file:
		file.write("\n".join(data))



X_train_treated = pretreatment(dataset_train)
random.shuffle(X_train_treated)
write_into_file("X_train.txt", X_train_treated)

In [26]:
model = fasttext.train_supervised(input="X_train.txt")

Read 5M words
Number of words:  121891
Number of labels: 2
Progress: 100.0% words/sec/thread: 2843174 lr:  0.000000 avg.loss:  0.400672 ETA:   0h 0m 0s


In [27]:
dataset_test = load_dataset("imdb", split="test")
X_test_treated = pretreatment(dataset_test)
random.shuffle(X_test_treated)
X_test_treated[:5]

Reusing dataset imdb (/home/victor/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


['__label__positive young elijah wood and joseph mazzello are outstanding in this excellent film about two boys who have promised to take care of their mother and how they cope when their new stepfather begins beating the younger boy the supporting cast around the boys is topnotch as well the script really gets inside the mind and heart of an imaginative child its hard to believe wood could grow up to look anything like tom hanks but thats nothing new in hollywood thats honestly my only criticism of the film',
 '__label__positive this is one of the best sequels around and a very good movie too even there were some mistakes but still i enjoyed it this time charles is not just fighting one or two muggers but a whole army of them death wish 3 has a lot great action scenes and i enjoyed it every single second of it director michael winner knows how to direct a good action movie like this and jimmy page providing the music with producers golan and globus still doing there thing and charles 

In [28]:
write_into_file("X_test.txt",X_test_treated)
model.test("X_test.txt")


(25000, 0.8768, 0.8768)

In [29]:
model.test("X_train.txt")

(25000, 0.90144, 0.90144)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_vallidation = train_test_split(X_train_treated)
write_into_file("X_train.txt",X_train)
write_into_file("X_val.txt", X_vallidation)

In [32]:
model2 = fasttext.train_supervised(input="X_train.txt",autotuneValidationFile="X_val.txt")
model2.test("X_test.txt")

Progress: 100.0% Trials:    9 Best score:  0.895680 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  104547
Number of labels: 2
Progress: 100.0% words/sec/thread: 1347112 lr:  0.000000 avg.loss:  0.047455 ETA:   0h 0m 0s  5.8% words/sec/thread: 1301743 lr:  0.080098 avg.loss:  0.472731 ETA:   0h 1m46s


(25000, 0.8912, 0.8912)

In [39]:
print("	   Model1 |  Model2 ")
def my_print(name: str, val1: str, val2 :str):
	print(f"{name}:  {val1}    | {val2}")
my_print("Epoch",model.epoch,model2.epoch)
my_print("Dim",model.dim,model2.dim) #Pas sur que ce soit intérréssant
my_print("Bucket",model.bucket,model2.bucket)
my_print("Loss function", model.loss,model2.loss)

	   Model1 |  Model2 
Epoch:  5    | 100
Dim:  100    | 92
Bucket:  0    | 4110692


In [47]:
Y_test = model2.predict(X_test_treated)

In [54]:
for i,j in enumerate(X_test_treated):
	print(i,j[9:16])

0 positiv
1 positiv
2 negativ
3 negativ
4 negativ
5 negativ
6 positiv
7 positiv
8 positiv
9 positiv
10 negativ
11 negativ
12 negativ
13 positiv
14 negativ
15 positiv
16 negativ
17 negativ
18 positiv
19 negativ
20 positiv
21 negativ
22 negativ
23 positiv
24 positiv
25 negativ
26 negativ
27 negativ
28 negativ
29 negativ
30 negativ
31 positiv
32 positiv
33 negativ
34 negativ
35 positiv
36 positiv
37 negativ
38 positiv
39 negativ
40 positiv
41 positiv
42 negativ
43 positiv
44 positiv
45 negativ
46 positiv
47 negativ
48 negativ
49 positiv
50 positiv
51 negativ
52 positiv
53 positiv
54 negativ
55 negativ
56 positiv
57 negativ
58 positiv
59 negativ
60 negativ
61 positiv
62 positiv
63 negativ
64 positiv
65 negativ
66 positiv
67 negativ
68 negativ
69 negativ
70 positiv
71 negativ
72 positiv
73 positiv
74 positiv
75 negativ
76 positiv
77 positiv
78 positiv
79 negativ
80 positiv
81 positiv
82 positiv
83 negativ
84 negativ
85 positiv
86 positiv
87 positiv
88 negativ
89 negativ
90 negativ
91 positi

In [58]:
print(X_test_treated[11],Y_test[0][11])
print(X_test_treated[15],Y_test[0][15])
print(X_test_treated[38],Y_test[0][38])

__label__negative 35 years after this was made castro still reigns unfortunately were left scratching our heads wondering how the dimwitted maniac played by scenerychewing jack palance made it as far as 1960 i stumbled back across this recently and was amused at noticing the incomparable sid haig and b movie favorite paul untouchables picerni among the rebels fleischer was obviously well past his prime when he directed this foolishness some of the lines are classic in a did he really say that kind of way the other thing i just noticed is that the score and the sound not the dialog are actually excellent  the only firstrate elements of the entire production so dont watch this to learn anything about history or acting but if you feel like watching this as a goof bring the beers and have some fun ['__label__positive']
__label__positive walter matthau and jack lemmon both of whom are sadly missed proved once again that they were a team dedicated to their craft of bringing hilarious moments